In [ ]:
from xcube.core.store import new_data_store
import os
import dask
from dask.distributed import Client, LocalCluster

In [ ]:
cluster = LocalCluster(
    n_workers=4,
    threads_per_worker=2,
    memory_limit="7GB",   # per worker
)
client = Client(cluster)

In [ ]:
store = new_data_store("stac-pc-ardc")

In [ ]:
from xcube_stac.utils import reproject_bbox

#bbox=[-6.3, 53.2, -3.2, 55.3]
#bbox=[-3.3,55.9,-2.3, 56.2] # Edinburgh
#bbox=[4.7,58.6,6.7, 60.6] # Stavanger
#bbox=[4.7,58.6,6.1, 59.6] # Stavanger
#bbox_wgs84 = [7.7, 55.5, 8.4, 56.1] #Esbjerg
#bbox_wgs84 = [4.3, 59.0, 5.7, 59.6] #north of Stavanger
bbox_wgs84 = [4.3, 58.0, 6.5, 59.6] #large Stavanger


crs_utm = "EPSG:32632"
bbox_utm = reproject_bbox(bbox_wgs84, "EPSG:4326", crs_utm)
bbox_utm

In [ ]:
store.list_data_ids()

In [ ]:
%%time
ds = store.open_data(
    "sentinel-2-l2a",
    #time_range=("2022-08-07", "2022-08-16"),
    time_range=("2023-06-07", "2023-06-16"),
    bbox=bbox_wgs84,
    spatial_res=10 / 111320,
    crs="EPSG:4326",
    asset_names=["B02", "B03", "B04", "B08", "SCL"],
    tile_size=(2048, 2048),
)
ds

In [ ]:
S3_USER_STORAGE_KEY = os.environ["S3_USER_STORAGE_KEY"]
S3_USER_STORAGE_SECRET = os.environ["S3_USER_STORAGE_SECRET"]
S3_USER_STORAGE_BUCKET = os.environ["S3_USER_STORAGE_BUCKET"]

In [ ]:
from xcube.core.store import new_data_store
team_store = new_data_store("s3", 
                       root=S3_USER_STORAGE_BUCKET, 
                       storage_options=dict(anon=False, 
                                            key=S3_USER_STORAGE_KEY, 
                                            secret=S3_USER_STORAGE_SECRET))

list(team_store.get_data_ids())

In [ ]:
%%time
team_store.write_data(ds,'sen2l2a_stavanger_large.zarr', replace=True)

In [ ]:
%%time
ds = team_store.open_data('sen2l2a_stavanger_large.zarr')
ds

In [ ]:
%%time
team_store.write_data(ds,"sen2l2a.levels", num_levels=5)